<a class="anchor" id="top"></a>
# Modeling Notebook
**Authors: Ainesh Pandey, Demian Gass, Gabriel Gilling**

In this notebook, we read in the prepped datasets and start modeling on our selected outcome variables.

## Table of Contents

[Step 1: Import Required Packages](#step-1) <br>
[Step 2: Read and Prepare Datasets](#step-2) <br>
[Step 3: Modeling](#step-3) <br>
[Step 4: Save Results for Analysis](#step-4) <br>

<a class="anchor" id="step-1"></a>

## Import Required Packages

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import pickle

from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

import time

from scripts.my_path import PATH

[Back to Top](#top)

<a class="anchor" id="step-2"></a>

## Read and Prepare Datasets

### Input Datasets

**Deltas Dataframe**: These are the primary inputs to our model.

In [2]:
with open('../Data/df_deltas.pkl', 'rb') as f:
    df_deltas = pickle.load(f)

# Normalize numeric features in deltas dataframe
for column in df_deltas.columns:
    if df_deltas[column].dtype == 'float64':
        df_deltas[column] = (df_deltas[column] - df_deltas[column].min()) / (df_deltas[column].max() - df_deltas[column].min())

print('Rows:',    df_deltas.shape[0])
print('Columns:', df_deltas.shape[1])
df_deltas.head()

Rows: 9289
Columns: 209


,PublicID,V1BA01_KG_delta_V2BA01_KG,V2BA01_KG_delta_V3BA01_KG,V1BA01_LB_delta_V2BA01_LB,V2BA01_LB_delta_V3BA01_LB,V2BA02b1_delta_V3BA02b1,V2BA02a1_delta_V3BA02a1,V2BA02b2_delta_V3BA02b2,V2BA02a2_delta_V3BA02a2,V1CA04_delta_V3CA04,...,U2BC03a_delta_U3BC03a,U2BA04_delta_U3BA04,U2BB04_delta_U3BB04,U2BB01_delta_U3BB01,U2BB03_delta_U3BB03,U2BA02_DY_delta_U3BA02_DY,U2BC01_delta_U3BC01,U2BA02_WK_delta_U3BA02_WK,U2BB02_delta_U3BB02,U2BC03c_delta_U3BC03c
0,00001U,0.522239,0.663086,0.512680,0.516508,0.339237,0.440188,0.514788,0.5745,0-3,...,S-S,0-0,0.580346,0-0,0-0,0.502593,0-0,0.453007,0.519531,S-S
1,00004O,0.522239,0.663086,0.472393,0.482693,0.336700,0.601595,0.514788,0.5745,3-3,...,S-S,2-2,0.580346,1-1,2-2,0.499938,2-2,0.585123,0.541750,S-S
2,00007I,0.522239,0.663086,0.510677,0.521473,0.395469,0.431082,0.514788,0.5745,3-3,...,S-S,2-2,0.580346,1-1,2-2,0.666791,2-2,0.418275,0.513992,S-S
3,00008G,0.522239,0.663086,0.563604,0.554183,0.331190,0.447863,0.514788,0.5745,3-2,...,S-S,2-2,0.580346,1-1,2-2,0.499814,2-2,0.519989,0.547669,S-S
4,00015J,0.522239,0.663086,0.530073,0.483132,0.421487,0.579686,0.514788,0.5745,1-1,...,S-S,2-2,0.580346,1-1,2-2,0.583240,2-2,0.316562,0.602954,S-S


**Covariates Dataframe**: We will adjust our models for selected demographic and socio-economic variables.

In [3]:
with open('../Data/df_covariates.pkl', 'rb') as f:
    df_covariates = pickle.load(f)
    
# Normalize numeric features in covariates dataframe
for column in df_covariates.columns:
    if df_covariates[column].dtype == 'float64':
        df_covariates[column] = (df_covariates[column] - df_covariates[column].min()) / (df_covariates[column].max() - df_covariates[column].min())

print('Rows:',    df_covariates.shape[0])
print('Columns:', df_covariates.shape[1])
df_covariates.head()

Rows: 9289
Columns: 17


,GAwks_screen,Age_at_V1,eRace,BMI,Education,GravCat,SmokeCat1,SmokeCat2,Ins_Govt,Ins_Mil,Ins_Comm,Ins_Pers,Ins_Othr,V1AF14,V1AG01,V1AG11,PublicID
0,0.750,0.50000,7,0.265029,6.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,11,1,2,00001U
1,0.750,0.25000,6,0.180132,3.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,5,2,2,00004O
2,0.625,0.18750,5,0.147336,3.0,3.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,4,1,1,00007I
3,0.625,0.53125,5,0.239248,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,10,1,2,00008G
4,0.750,0.59375,5,0.114749,6.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,12,1,2,00015J


**Base Dataframe**: Combine the inputs (the deltas and the covariates) into the base dataset.

In [4]:
df_base = df_deltas.merge(df_covariates, on='PublicID', how='inner')

display(df_base.shape)
df_base.head()

(9289, 225)

,PublicID,V1BA01_KG_delta_V2BA01_KG,V2BA01_KG_delta_V3BA01_KG,V1BA01_LB_delta_V2BA01_LB,V2BA01_LB_delta_V3BA01_LB,V2BA02b1_delta_V3BA02b1,V2BA02a1_delta_V3BA02a1,V2BA02b2_delta_V3BA02b2,V2BA02a2_delta_V3BA02a2,V1CA04_delta_V3CA04,...,SmokeCat1,SmokeCat2,Ins_Govt,Ins_Mil,Ins_Comm,Ins_Pers,Ins_Othr,V1AF14,V1AG01,V1AG11
0,00001U,0.522239,0.663086,0.512680,0.516508,0.339237,0.440188,0.514788,0.5745,0-3,...,1.0,2.0,2.0,2.0,1.0,2.0,2.0,11,1,2
1,00004O,0.522239,0.663086,0.472393,0.482693,0.336700,0.601595,0.514788,0.5745,3-3,...,2.0,2.0,2.0,2.0,1.0,2.0,2.0,5,2,2
2,00007I,0.522239,0.663086,0.510677,0.521473,0.395469,0.431082,0.514788,0.5745,3-3,...,1.0,1.0,1.0,2.0,2.0,2.0,2.0,4,1,1
3,00008G,0.522239,0.663086,0.563604,0.554183,0.331190,0.447863,0.514788,0.5745,3-2,...,2.0,2.0,2.0,2.0,1.0,1.0,2.0,10,1,2
4,00015J,0.522239,0.663086,0.530073,0.483132,0.421487,0.579686,0.514788,0.5745,1-1,...,2.0,2.0,2.0,2.0,1.0,2.0,2.0,12,1,2


### Targets Datasets

**Target Dataframe**: The target variables we will be predicting on.

In [5]:
with open('../Data/df_targets.pkl', 'rb') as f:
    df_targets = pickle.load(f)

print('Rows:',    df_targets.shape[0])
print('Columns:', df_targets.shape[1])
df_targets.head()

Rows: 9289
Columns: 19


,PEgHTN,ChronHTN,CMAD01a,CMAD01b,CMAD01c,CMAD01d,CMAD01e,CMAD01f,CMAD01g,CMAD01h,CMAE04a1c,CMAE04a2c,CMAE04a3c,CMAE04a4c,CMAE04a5c,Stillbirth,Miscarriage,Termination,PublicID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00001U
1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00004O
2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00007I
3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00008G
4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00015J


### Auxiliary Datasets

#### Variables Dictionary

In [6]:
variables_df = pd.read_excel(PATH + '/nuMoM2b_Codebook_NICHD Data Challenge.xlsx',
                              sheet_name='nuMoM2b_Variables',
                              header=1,
                              usecols=['Variable Name', 'Variable Label', 'Variable Type', 'Variable Code List\n(if Coded)'],
                              engine='openpyxl')
variables_df.columns = ['Variable Name', 'Variable Label', 'Variable Type', 'Variable Code List']

variables_df.head()

,Variable Name,Variable Label,Variable Type,Variable Code List
0,PublicID,Public nuMoM2b ID,Character,NaN
1,A02_Complete,(A02) Data entry status,Character,NaN
2,A02_Complete_1,(A02) Data entry status,Character,NaN
3,A02_Status,(A02) Validation status,Character,NaN
4,A02_Status_1,(A02) Validation status,Character,NaN


### Results Datasets

**Features Importance Dictionary**: We will store the feature importances for each target variable in this dictionary.

In [7]:
feature_results_dict = {}

**Model Results Dictionary**: We will store the model results for each target variables in this dictionary.

In [8]:
model_results_dict = {}

[Back to Top](#top)

<a class="anchor" id="step-3"></a>

## Modeling

We will try the following modeling approaches, as explained in our README file.
- `Logistic Regression`
- `Random Forest`
- `Light Gradient-Boosted Model`

### Helper Functions

These will be used in the master function.

In [9]:
# The master function will use this helper function to prep the data for each target variable
def prep_and_split_data(target, df_base, df_targets):
    
    try:
        # append target feature to base dataframe
        df = df_base.merge(df_targets[['PublicID', target]], on='PublicID').drop('PublicID', axis=1)

        # drop rows missing the output feature
        # print('  Num rows before dropping:', df.shape[0])
        # print('  Num missing values:', df[target].isna().sum())
        df = df.dropna(subset=[target])
        # print('  Num rows after dropping:', df.shape[0])

        # split into X and y
        X = df.drop([target], axis = 1)
        y = df[target]

        # drop correlated features
        # print('  Num columns before dropping correlated features:', X.shape[1])
        corr = X.corr()
        upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
        to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
        X.drop(to_drop, axis=1, inplace=True)
        # print('  Num columns after dropping correlated features:', X.shape[1])

        # one-hot encode X
        X_dummied = pd.get_dummies(X, prefix_sep='__')
        # print('  Num columns after one-hot encoding:', X_dummied.shape[1])
        
        # train/test split with NearMiss undersampling
        X_dummied, y = NearMiss(version=3, n_neighbors_ver3=3).fit_resample(X_dummied, y)
        # print('  Num rows after Near-Miss undersampling:', X_dummied.shape[0])
        print('  Dataframe shape after cleaning:', str(X_dummied.shape))

        # save dataframe for access during analysis phase
        filename = 'Target Pickles/df_' + target + '.pkl'
        with open(filename, 'wb') as f:
            pickle.dump(pd.concat([X_dummied, y], axis=1), f)
        # print('  Dataframe saved:', filename)

        return train_test_split(X_dummied, y, test_size=0.3, random_state=42, stratify = y)
    
    except:
        pass

In [10]:
# The master function will use this function to initialize the feature_results dataframe for each target variable
def initialize_feature_results(X_dummied):
    
    # initialize the feature results dataframe
    feature_results = pd.DataFrame(columns=['Feature', 'Variable1 Name', 'Variable1 Desc', 'Variable2 Name', 'Variable2 Desc',
                                            'LogR_FeatureImportance', 'RanF_FeatureImportance', 'LGBM_FeatureImportance'])
    
    # populate the "Feature" column
    feature_results['Feature'] = X_dummied.columns

    # extract the individual features from the delta columns, nan for second feature if not a delta column
    feature_split = [x.split('__')[0].split('_delta_') for x in feature_results['Feature']]
    for x in feature_split:
        if(len(x) == 1): x.append(np.nan)
    feature_results['Variable1 Name'] = [x[0] for x in feature_split]
    feature_results['Variable2 Name'] = [x[1] for x in feature_split]

    # extract the feature labels for all features
    feature_results['Variable1 Desc'] = feature_results[['Variable1 Name']].merge(variables_df[['Variable Name', 'Variable Label']], how='left',
                                                                                  left_on='Variable1 Name', right_on='Variable Name')['Variable Label']
    feature_results['Variable2 Desc'] = feature_results[['Variable2 Name']].merge(variables_df[['Variable Name', 'Variable Label']], how='left',
                                                                                  left_on='Variable2 Name', right_on='Variable Name')['Variable Label']

    # extract the feature labels for all features
    feature_results['Variable1 Desc'] = feature_results[['Variable1 Name']].merge(variables_df[['Variable Name', 'Variable Label']], how='left',
                                                                                  left_on='Variable1 Name', right_on='Variable Name')['Variable Label']
    feature_results['Variable2 Desc'] = feature_results[['Variable2 Name']].merge(variables_df[['Variable Name', 'Variable Label']], how='left',
                                                                                  left_on='Variable2 Name', right_on='Variable Name')['Variable Label']
    
    return feature_results

### Functions for Modeling Approaches

***
**Logistic Regression**
***

In [11]:
# create function that runs and tunes logistic regression, outputs results
def model_logisticregression_tuned(X_train, y_train, X_test, y_test):
    
    # create parameter grid to fine-tune model
    param_grid = { 'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 10.0] }
    
    # run the cross-validated grid search to identify the best parameters for the model
    CV_clf = GridSearchCV(estimator=LogisticRegression(penalty='l1', solver='liblinear', random_state=42),
                          scoring='f1', param_grid=param_grid, n_jobs=-1, verbose=0).fit(X_train, y_train)
    
    # extract the best parameters, as selected by the grid search
    best_params = CV_clf.best_params_
    best_C = best_params['C']
    
    # create the final RandomForestClassifier
    best_clf = LogisticRegression(random_state=42, C=best_C).fit(X_train, y_train)
    
    # predict on the test set
    y_pred = np.round(best_clf.predict(X_test))
    
    # Create dataframe for parameters and feature importances
    features_df = pd.DataFrame()
    features_df['Parameter'] = X_train.columns.to_list()
    features_df['Feature Importance'] = best_clf.coef_[0]
    
    # return the features dataframe and a classification report
    return [features_df, classification_report(y_test, y_pred, output_dict = True)]

***
**Random Forest**
***

In [12]:
# create function that runs and tunes random forest, outputs results
def model_randomforest_tuned(X_train, y_train, X_test, y_test):
    
    # create parameter grid to fine-tune model
    param_grid = { 
        'n_estimators': range(100, 600, 100),
        'max_features': ['auto', 'log2', 0.2, 0.25, 0.33, 0.5],
        'max_depth' : [None, 4, 6, 8],
        'criterion' : ['gini', 'entropy']
    }
    
    # run the cross-validated grid search to identify the best parameters for the model
    CV_rfc = GridSearchCV(estimator=RandomForestClassifier(random_state=42), scoring='f1',
                          param_grid=param_grid, n_jobs=-1, verbose=0).fit(X_train, y_train)
    
    # extract the best parameters, as selected by the grid search
    best_params = CV_rfc.best_params_
    best_n_estimators = best_params['n_estimators']
    best_max_features = best_params['max_features']
    best_max_depth = best_params['max_depth']
    best_criterion = best_params['criterion']
    
    # create the final RandomForestClassifier
    best_rfc = RandomForestClassifier(random_state=42,
                                      max_features=best_max_features,
                                      n_estimators=best_n_estimators,
                                      max_depth=best_max_depth,
                                      criterion=best_criterion).fit(X_train, y_train)
    
    # predict on the test set
    y_pred = best_rfc.predict(X_test)
    
    # Create dataframe for parameters and feature importances
    features_df = pd.DataFrame()
    features_df['Parameter'] = X_train.columns.to_list()
    features_df['Feature Importance'] = best_rfc.feature_importances_
    
    # return the features dataframe and a classification report
    return [features_df, classification_report(y_test, y_pred, output_dict = True)]

***
**Light GBM**
***

In [13]:
# create function that runs lgbm, outputs results
def model_lgbm_tuned(X_train, y_train, X_test, y_test):
    
    # create parameter grid to fine-tune model
    param_grid = {
        'colsample_bytree': [0.8, 1.0],
        'max_depth': [15, 20, -1],
        'num_leaves': [10, 20, 31],
        'reg_alpha': [0, 0.5, 1.0],
        'reg_lambda': [0, 0.5, 1.0],
        'min_split_gain': [0, 0.2, 0.4],
        'subsample': [0.8, 1.0]
    }
    
    # run the cross-validated grid search to identify the best parameters for the model
    CV_lgb = GridSearchCV(estimator=LGBMClassifier(random_state=42), scoring='f1',
                          param_grid=param_grid, n_jobs=-1, verbose=0).fit(X_train, y_train)
    
    # extract the best parameters, as selected by the grid search
    best_params = CV_lgb.best_params_
    best_colsample_bytree = best_params['colsample_bytree']
    best_max_depth = best_params['max_depth']
    best_num_leaves = best_params['num_leaves']
    best_reg_alpha = best_params['reg_alpha']
    best_reg_lambda = best_params['reg_lambda']
    best_min_split_gain = best_params['min_split_gain']
    best_subsample = best_params['subsample']
    
    # create the final LGBMClassifier
    best_lgb = LGBMClassifier(random_state=42,
                              colsample_bytree=best_colsample_bytree,
                              max_depth=best_max_depth,
                              num_leaves=best_num_leaves,
                              reg_alpha=best_reg_alpha,
                              reg_lambda=best_reg_lambda,
                              min_split_gain=best_min_split_gain,
                              subsample=best_subsample).fit(X_train, y_train)
    
    # predict on the test set
    y_pred = best_lgb.predict(X_test)

    # create dataframe for parameters and feature importances
    features_df = pd.DataFrame()
    features_df['Parameter'] = best_lgb.feature_name_
    features_df['Feature Importance'] = best_lgb.feature_importances_

    # return the features dataframe and a classification report
    return [features_df, classification_report(y_test, y_pred, output_dict = True)]

### Master Modeling Function

In [14]:
def run_models_tuned(target, df_base, df_targets):
    try:
        print('Modeling for target =', target)

        # call the prep_and_split_data() helper function to extract the training and test sets
        X_train, X_test, y_train, y_test = prep_and_split_data(target, df_base, df_targets)

        # initialize model results dictionary and feature results dataframe for selected target variable
        model_results = {}
        feature_results = initialize_feature_results(X_train)

        # Run the logistic regression model, extract the results
        print('  Training and tuning Logistic Regression model...')
        t0 = time.time()
        logr_features, model_results['LogR'] = model_logisticregression_tuned(X_train, y_train, X_test, y_test)
        feature_results['LogR_FeatureImportance'] = feature_results[['Feature']].merge(logr_features, how='inner',
                                                                                       left_on='Feature', right_on='Parameter')['Feature Importance']
        t1 = time.time()
        print('    Done running Logistic Regression model in', str(int((t1-t0)/60)), 'mins and', str(int((t1-t0)%60)), 'secs')

        # Run the random forest model, extract the results
        print('  Training and tuning Random Forest model...')
        t0 = time.time()
        ranf_features, model_results['RanF'] = model_randomforest_tuned(X_train, y_train, X_test, y_test)
        feature_results['RanF_FeatureImportance'] = feature_results[['Feature']].merge(ranf_features, how='inner',
                                                                                       left_on='Feature', right_on='Parameter')['Feature Importance']
        t1 = time.time()
        print('    Done running Random Forest model in', str(int((t1-t0)/60)), 'mins and', str(int((t1-t0)%60)), 'secs')

        # Run the LGBM model, extract the results
        print('  Training and tuning LGBM model...')
        t0 = time.time()
        lgbm_features, model_results['LGBM'] = model_lgbm_tuned(X_train, y_train, X_test, y_test)
        feature_results['LGBM_FeatureImportance'] = feature_results[['Feature']].merge(lgbm_features, how='inner',
                                                                                       left_on='Feature', right_on='Parameter')['Feature Importance']
        t1 = time.time()
        print('    Done running LGBM model in', str(int((t1-t0)/60)), 'mins and', str(int((t1-t0)%60)), 'secs')

        # Save results back to dictionaries
        model_results_dict[target]   = model_results
        feature_results_dict[target] = feature_results
        print('Modeling successful!')
        print()
        
    except:
        print('Modeling failed, due to error')
        print()
        pass

In [17]:
for target in df_targets.columns.drop('PublicID').sort_values():
    run_models_tuned(target, df_base, df_targets)

Modeling for target = CMAD01a
  Dataframe shape after cleaning: (168, 2407)
  Training and tuning Logistic Regression model...
    Done running Logistic Regression model in 0 mins and 1 secs
  Training and tuning Random Forest model...
    Done running Random Forest model in 1 mins and 58 secs
  Training and tuning LGBM model...
    Done running LGBM model in 6 mins and 23 secs
Modeling successful!

Modeling for target = CMAD01b
  Dataframe shape after cleaning: (156, 2407)
  Training and tuning Logistic Regression model...
    Done running Logistic Regression model in 0 mins and 0 secs
  Training and tuning Random Forest model...
    Done running Random Forest model in 1 mins and 55 secs
  Training and tuning LGBM model...
    Done running LGBM model in 5 mins and 41 secs
Modeling successful!

Modeling for target = CMAD01c
  Dataframe shape after cleaning: (162, 2407)
  Training and tuning Logistic Regression model...
    Done running Logistic Regression model in 0 mins and 0 secs
  T

<a class="anchor" id="step-4"></a>

## Save Results for Analysis

In [18]:
with open('../Results/model_results_dict.pkl', 'wb') as f:
    pickle.dump(model_results_dict, f)

In [19]:
with open('../Results/feature_results_dict.pkl', 'wb') as f:
    pickle.dump(feature_results_dict, f)

[Back to Top](#top)